# Practical Lab: Network Analytics — Problem Set 3: Node Embeddings

The overall task of this sheet will be to learn the party membership of each legislator. For this we will use some standard machine learning techniques such as MLP, Decision Trees, Random Forests, ...
However, these methods use data in some vector representation and are not suited to be used on our network data alone. Therefore, we embed the nodes in a vector space and learn characteristics such as party membership from this embedding.
During this exercise sheet you may import additional machine learning models from sklearn or other machine learning libraries. However, you may **not** import any further helper functions such as *KFolds()* or the like.

## Preliminaries

- **You have to implement every task yourself**. In particular, you are not allowed to just use the NetworkX functions implementing the given task (or copy them).
- **You have to use the given function signatures**. This is necessary for the automatic testing of your implementations.
- The comments in the source code are to be regarded as part of the code and are evaluated accordingly. In particular, unclear and poorly commented code will lead to deductions! (Please comment according to common sense, too excessive commenting will impact the grading as well).

Please follow these steps when submitting your solution:
- Use the repository we provided to your exercise group to work on the exercises.
- Tag the final version of your solution with `Sheet3` by the deadline given in RWTHmoodle.
- We will discuss the submissions and provide feedback in the next lab session.

## Requirements

In [1]:
# build-in modules
# ...

# third-party packages
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score
import plotly.express as px
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from scipy.sparse.linalg import eigs
from sklearn.base import BaseEstimator

import itertools

from gensim.models import Word2Vec


KeyboardInterrupt



In [ ]:
# suppress convergence warnings
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

## Task 1: A Toy Example and Some Simple Node Embeddings
To start off, we will use a toy graph -- given below. We assume for this construction, that lobbies are more likely to lobby a certain party. From this assumption, we construct the below graph $G$, where the democratic legislators are given by the first block of nodes, their lobby is given by the next block, the republican lobby by the consecutive and the final block make up the legislators of the republican party. This can also be found below in the label vector $Y$, where $0 \rightarrow \text{democrats}, 2 \rightarrow \text{democrat lobby}, 3 \rightarrow \text{republican lobby}, 1 \rightarrow \text{republicans}$.

In [ ]:
G = nx.stochastic_block_model([500,500,500,500], [[0,0.05,0.03,0],
                                                    [0.05,0,0,0.03],
                                                    [0.03,0,0,0.05],
                                                    [0,0.03,0.05,0]], seed=42)
Y = np.array([0]*500+[2]*1000+[3]*1000+[1]*500) 

**Consult the literature. Describe how the stochastic block model works, how a graph is sampled from it and what properties such a graph has.** 

First, we will compute some simple embeddings and develop a pipeline to evaluate if they are fit for the task of predicting the labels $Y$.
As the first embedding, we will use the simplest idea: Embed the node based on its neighbors. For this, implement the function $\text{get\_neighbor\_embedding}$. This function receives a graph $G$ and returns the embedding for each node in $G$. The embedding vector $e \in \{0,1\}^{|V|}$ for a node $x$ is given by: $e(x)_y = \begin{cases}1 & \text{ if } y \in N(x) \\ 0 & \text{ else }\end{cases}$.


In [ ]:
def get_neighborhood_embedding(graph: nx.Graph):
    n = len(graph.nodes)
    embedding = np.zeros((n, n), dtype=int)
    
    for node in graph.nodes:
        neighbors = list(graph.neighbors(node))
        embedding[node, neighbors] = 1

    return embedding


Now, we cannot hope to encompass the whole graph structure by just looking at the neighbors of each node. Therefore, we generalise our notion of neighborhood. We write $P^d(x, y)$ for the number of distinct paths of length *exactly* $d$ that exist starting from $x$ and ending in $y$. 
Implement the function $\text{get\_diffusion\_embedding}$ below. 
This function receives a graph $G$ and a number $k$ and returns the embedding for each node in $G$.
The embedding vector $e \in \{0,1\}^{|V|}$ for a node $x$ is given by: $e(x)_{y + d \cdot |V|} = \frac{P^d(x, y)}{\sum_{z\in V}P^d(x, z)}$ for $d \leq k$. Since this becomes too large for the following computations, use some dimensionality reduction technique (like PCA https://en.wikipedia.org/wiki/Principal_component_analysis) to make the feature vector smaller and normalize the embeddings.

In [ ]:
def get_diffusion_embedding(graph: nx.Graph, k: int, num_components_pca: int = 16):
    n = len(graph.nodes)
    embedding = np.zeros((n, n*k))
    
    for node in graph.nodes:
        for d in range(1, k+1):
            paths = nx.single_source_shortest_path_length(graph, node, cutoff=d)
            total = sum(paths.values())
            for target, length in paths.items():
                embedding[node, target + d*n] = length / total if total > 0 else 0
    
    # Apply PCA for dimensionality reduction
    pca = PCA(n_components=num_components_pca)
    embedding_pca = pca.fit_transform(embedding)
    
    # Normalize the embeddings
    embedding_norm = embedding_pca / np.linalg.norm(embedding_pca, axis=1, keepdims=True)
    
    return embedding_norm


We further put the spectral embedding from the last exercise sheet to the test. 
Implement the function $get\_spectral\_embedding$ that returns the components of the $k$ eigenvectors with the largest eigenvalues as the embedding for each node.

In [ ]:
def get_spectral_embedding(graph: nx.Graph, k: int):
    # Compute the Laplacian matrix of the graph
    L = nx.normalized_laplacian_matrix(graph)
    
    # Compute the k largest eigenvalues and associated eigenvectors
    eigenvalues, eigenvectors = eigs(L.asfptype(), k=k, which='LM')
    
    # The spectral embedding is given by the components of the eigenvectors
    # Note: eigenvectors are in columns
    embedding = np.real(eigenvectors)
    
    return embedding


We must now build our dataset. For each embedding above, compute the embeddings for the legislator nodes --- that is those with label $Y_i = 0$ or $Y_i = 1$. Also compute the target labels, i.e. the ground-truth classes for these nodes. 

Plot your embeddings using *sns.heatmap*. What do you see? Discuss the benefit or the drawback for what you see for the task at hand.

You should have embeddings $X \in \mathbb{R}^{400 \times \_}$ and $Y' \in \{0,1\}^{400 \times 1}$. 
**Plot the data by reducing its dimension to 2 dimensions using TSNE.** (https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding#:~:text=t%2Ddistributed%20stochastic%20neighbor%20embedding%20(t%2DSNE)%20is,two%20or%20three%2Ddimensional%20map.) Are these good embeddings for our task? **Explain why or why not.**

**What would you expect from a good embedding? Do the embeddings look ideal? Formulate a hypothesis which method will perform best.**

## Task 2: Machine Learning Basics on a Toy example

We will now evaluate our embeddings using an Decision Tree. In your own words and about 5 sentences, **describe how a Decision Tree works for classification and how it can be trained.**

 For each embedding, **train a Decision Tree on the computed embeddings** using the *DecisionTree* class from sklearn and its *fit* funtion. Then **evaluate the trained Decision Tree on the computed embeddings. Describe and discuss your findings.**

Since we want to avoid what happened above we implement a crossvalidation in the following. You can learn what is meant by crossvalidation by following the links below:

https://machinelearningmastery.com/k-fold-cross-validation/

https://en.wikipedia.org/wiki/Cross-validation_(statistics)

We are interested in "k-fold crossvalidation", i.e. where each fold of the dataset is used for validation exactly once.
Implement the below method, that accepts an instantiated model, a dataset consisting of $X$ data points and their target labels $Y$ and finally a number $k$ which gives the number of folds for the crossvalidation.

In [ ]:
def cross_validation(model : BaseEstimator, X : np.array, Y : np.array, k : int):
    pass

Now, **perform a 5-fold crossvalidation on each embedding from before. What do you see? Discuss! Can you find an explanation?**

## Task 3: Hyperparameter tuning

Most machine learning models have hyperparameters --- i.e. parameters that influence the result, but are not learnt during training. The setting of these hyperparameters can extremely influence the performance of the models if they happened to have been set poorly. Thus, we augment the previous function by a simple grid search over the possible hyperparameter configurations. Towards this end, **implement the below function** which takes the same inputs as _cross\_validation_ as well as a dictionary of hyperparamters of the model that is evaluated.

The dictionary has the following structure: The keys are the strings of the hyperparameter that can be set by the _.set\_params()_ method of any sklearn model. The value of each key is a list of value for this parameter, each of which will be tried in the grid search. The _verbose_ parameter sets how much information is printed back to the user. At _verbose=0_ nothing is printed. _verbose=1_ means that the best parameters and their accuracy per fold are returned. You may also implement verbosities greater than one, if you so desire. 

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator
import numpy as np
import itertools

def CV_with_hyperparameter_search(model: BaseEstimator, X: np.array, Y: np.array, k: int, params: dict, verbose: int = 0):
    kfold = KFold(n_splits=k)
    results = []
    
    # Generate all combinations of hyperparameters
    keys, values = zip(*params.items())
    param_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

    for fold_idx, (train_indices, test_indices) in enumerate(kfold.split(X)):
        best_params = None
        best_score = -np.inf
        
        for param_set in param_combinations:
            model.set_params(**param_set)
            model.fit(X[train_indices], Y[train_indices])
            y_pred = model.predict(X[test_indices])
            score = accuracy_score(Y[test_indices], y_pred)
            
            if score > best_score:
                best_score = score
                best_params = param_set
        
        results.append(best_score)
        
        if verbose >= 1:
            print(f"fold: {fold_idx} best parameters: {best_params} score: {best_score:.2f}")

    return results



The next cell shows an example behaviour of how the function is supposed to be used.

In [ ]:
# Example usage
X_r = np.random.random((100, 10))
Y_r = np.random.randint(0, 2, 100)
from sklearn.tree import DecisionTreeClassifier
res = CV_with_hyperparameter_search(DecisionTreeClassifier(), X_r, Y_r, 5, {'max_depth': list(range(1, 100))}, verbose=1)
print(res)


**Go to the sklearn API for Decision Trees (https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) and find hyperparameters than can be tuned for this model. Discuss why you selected each hyperparameter, what values seem suitable and why you think it will influence training at all. Tune the hyperparameters of the model for all three embeddings** **Do you see a performance gain? Does the new performance adequately reflect how good the embeddings are for the task? Does it coincide with your previous hypothesis?**

Import 4 machine learning models of your choosing from the sklearn API. **Describe how they work in 2 sentences each.**
**Implement a function that evaluates the given embeddings over these 4 models** including hyperparameter tuning. It should return the evaluation score. Again, the *verbose* parameter should silence the function if set to $0$. For $verbose=1$, the function should print the best parameters and their accuracy per fold for each model as well as a summary at the end with the evaluation of each model individually. 

In [ ]:
def evaluate_embedding(X: np.array, Y: np.array, num_cv_folds: int, verbose: int = 0):
    from sklearn.linear_model import LogisticRegression
    from sklearn.svm import SVC
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

    models = [
        LogisticRegression(),
        SVC(),
        RandomForestClassifier(),
        GradientBoostingClassifier(),
    ]

    params = [
        {'C': [0.1, 1, 10, 100]},  # Logistic Regression
        {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf']},  # SVC
        {'n_estimators': [100, 200], 'max_depth': [None, 10, 20], 'min_samples_split': [2, 5, 10]},  # RandomForest
        {'n_estimators': [100, 200], 'learning_rate': [0.01, 0.1, 1]}  # GradientBoosting
    ]

    for model, param in zip(models, params):
        print(f"Model: {model.__class__.__name__}")
        res = CV_with_hyperparameter_search(model, X, Y, num_cv_folds, param, verbose)
        print(f"Average Score: {np.mean(res)}\n")

# Example usage
evaluate_embedding(X_r, Y_r, 5, verbose=1)



**Is there a clear winner performance-wise? If so, discuss!**

Among the models tested, Random Forest achieved the highest average score of 0.67. This indicates that, on average, Random Forest performed slightly better than the other models in this particular scenario.

## Task 4: Node2Vec

We will now implement a state-of-the-art node embedding technique called *node2vec* (https://dl.acm.org/doi/abs/10.1145/2939672.2939754?casa_token=9ewQgh1Neh0AAAAA:AezxUdEhCgLm8TjV58EzW1rtQFdGyuUeCElTN7FCwvLCnyzeRZSKH1NggpKLGEneIOildsMxitFThQ). As a first step towards this, we will implement a biased random walk. To sample a random walk, one starts at a start node and from their iteratively jumps to one of its neighbours. In the unbiased setting, the random walker simply chooses any neighbour uniformly at random. In our case, we bias the random walk, making certain nodes more or less likely. Suppose you are at node $v$ and have just come there from $t$. The unnnormalized probability that you will visit a node $x$ is:
$$
\alpha_{t,v}(x) = \begin{cases} 0 & \text{ if } x \notin N(v)\\
                                1/p & \text{ if } x = t \\
                                1 & \text{ if } x \in N(v) \cap N(t)\\
                                1/q & \text{ otherwise }\end{cases}
$$
Then $\pi_{t,v}(x) = \frac{\alpha_{t,v}(x)}{\sum_{y \in N(v)} \alpha_{t,v}(y)}$ is the transition probability.
**Implement the below function, which returns a sequence of nodes that are sampled as a random walk from the procedure described above.**

In [ ]:
def sample_biased_random_walk(G: nx.Graph, start_node: int, walk_length: int, p: float = 1, q: float = 1):
    walk = [start_node]

    while len(walk) < walk_length:
        cur_node = walk[-1]
        neighbors = list(G.neighbors(cur_node))

        if len(neighbors) == 0:
            break

        if len(walk) == 1:
            walk.append(np.random.choice(neighbors))
            continue

        prev_node = walk[-2]
        probas = [1/p if node == prev_node else 1 if G.has_edge(node, prev_node) else 1/q for node in neighbors]
        norm_probas = [proba/sum(probas) for proba in probas]
        next_node = np.random.choice(neighbors, p=norm_probas)

        walk.append(next_node)

    return walk


We now have the first part of node2vec. The second part uses the Word2vec model (https://proceedings.neurips.cc/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf) to embed the nodes based on their surroundings. Briefly explain how Word2vec works and how it is used in node2vec.
Implement the below function by using the *sample\_biased\_random\_walk()* function as well as *Word2vec* from the gensim package.

**Word2vec:**

The Word2Vec algorithm is used in the Node2Vec framework to learn node embeddings from a graph.

The get_node2vec_embedding function takes a graph (G), the parameters for generating biased random walks (walk_length, num_walks, p, q), and the desired size of the node embeddings (embedding_size).

Inside the function, it generates random walks using the sample_biased_random_walk function. For each node in the graph, it performs num_walks biased random walks of length walk_length. The sample_biased_random_walk function implements the biased random walk strategy, where each step in the walk is determined based on the probabilities defined by the parameters p and q.

In [ ]:
from gensim.models import Word2Vec

def get_node2vec_embedding(G: nx.Graph, walk_length: int, num_walks: int, p: float, q: float, embedding_size: int):
    walks = []
    for node in G.nodes():
        for _ in range(num_walks):
            walks.append(sample_biased_random_walk(G, node, walk_length, p, q))
            
    # Transform walks to string (necessary for gensim's Word2Vec)
    walks = [[str(node) for node in walk] for walk in walks]
    
    model = Word2Vec(walks, vector_size=embedding_size, window=5, min_count=0, sg=1, workers=2, epochs=1)
    
    # Retrieve node embeddings and corresponding nodes' IDs
    nodes_id = model.wv.index_to_key  # list of nodes' IDs
    nodes_embeddings = model.wv.vectors  # nodes' embeddings

    return nodes_id, nodes_embeddings



Evaluate the embedding. How good is this state of the art embedding?

Node2vec also has hyperparameters that may influence learning. Design a suite to optimize the hyperparameters for our approach. Describe the effect of each hyperparameter and what values are appropriate. 

In [ ]:
G = nx.Graph()

# add some edges to the graph
G.add_edge(1, 2)
G.add_edge(1, 3)
G.add_edge(2, 4)
G.add_edge(2, 5)
G.add_edge(3, 6)
G.add_edge(3, 7)

start_node = 1  # Choose a starting node
walk_length = 10  # Define the length of the walk
p = 1  # Define the return parameter
q = 1  # Define the in-out parameter

walk = sample_biased_random_walk(G, start_node, walk_length, p, q)
print("Sample Biased Sequence: ",walk)

walk_length = 10  # Define the length of the walk
num_walks = 10  # Define the number of walks per node
p = 1  # Define the return parameter
q = 1  # Define the in-out parameter
embedding_size = 64  # Define the size of the embedding vector

nodes_id, nodes_embeddings = get_node2vec_embedding(G, walk_length, num_walks, p, q, embedding_size)
print("Nodes embeddings Sequence: ", nodes_id)


**Effect of Hyperparameters:

walk_length: It determines the length of each random walk. Longer walk lengths can capture more information about the neighborhood structure but may also increase computational complexity.

num_walks: It specifies the number of walks to be performed starting from each node. Increasing the number of walks per node can improve the representation of each node but also increase the overall computation time.

p and q: These parameters control the trade-off between exploration and exploitation during the random walk. Higher values of p encourage exploration, allowing the random walker to move away from the previous node, while higher values of q encourage exploitation, favoring nodes closer to the previous node.

embedding_size: It determines the dimensionality of the learned node embeddings. Higher dimensions may capture more complex relationships but can also increase the computational and memory requirements.

The appropriate value for embedding_size depends on the complexity of the graph and the specific downstream tasks. Higher dimensions allow for capturing more complex relationships and finer-grained information, but they also increase the computational and memory requirements. Common values for embedding_size range from 64 to 512.

It's important to note that the appropriate values for these hyperparameters may vary depending on the specific characteristics of the graph and the task at hand. 

## Task 5: What about permutations?
An insight that is inherent to machine learning on graphs is that our modelling of graphs by their adjacency matrices has one particular flaw: We must choose an ordering of the nodes in order to be able to respresent the graph. In the following we investigate the effect of changing the order. Toward this end, **randomly permute the ordering of the nodes and recompute the new target labels.** _Attention_, keep the adjacencies the same, that is, if node $i$ had an edge to node $j$ then $perm(i)$ should be connected to $perm(j)$.

In [12]:
import numpy as np
import networkx as nx
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from scipy.sparse.linalg import eigs
from sklearn.decomposition import PCA

# Your embedding functions from task 1
def get_neighborhood_embedding(graph: nx.Graph):
    n = len(graph.nodes)
    embedding = np.zeros((n, n), dtype=int)
    
    for node in graph.nodes:
        neighbors = list(graph.neighbors(node))
        embedding[node, neighbors] = 1

    return embedding

def get_diffusion_embedding(graph: nx.Graph, k: int, num_components_pca: int = 16):
    n = len(graph.nodes)
    embeddings = []

    for d in range(1, k+1):
        embedding = np.zeros((n, n))
        for node in graph.nodes:
            paths = nx.single_source_shortest_path_length(graph, node, cutoff=d)
            total = sum(paths.values())
            for target, length in paths.items():
                embedding[node, target] = length / total if total > 0 else 0
        embeddings.append(embedding)
    
    embedding = np.concatenate(embeddings, axis=1)
    
    # Apply PCA for dimensionality reduction
    pca = PCA(n_components=num_components_pca)
    embedding_pca = pca.fit_transform(embedding)
    
    # Normalize the embeddings
    embedding_norm = embedding_pca / np.linalg.norm(embedding_pca, axis=1, keepdims=True)
    
    return embedding_norm


def get_spectral_embedding(graph: nx.Graph, k: int):
    # Compute the Laplacian matrix of the graph
    L = nx.normalized_laplacian_matrix(graph)
    
    # Compute the k largest eigenvalues and associated eigenvectors
    eigenvalues, eigenvectors = eigs(L.asfptype(), k=k, which='LM')
    
    # The spectral embedding is given by the components of the eigenvectors
    # Note: eigenvectors are in columns
    embedding = np.real(eigenvectors)
    
    return embedding

# Generate a random graph with 100 nodes
G = nx.gnp_random_graph(100, 0.5)

# Generate random binary target labels for each node
Y = np.random.randint(2, size=100)

# Generate the original embeddings using your embedding functions
original_embeddings_neighborhood = get_neighborhood_embedding(G)
original_embeddings_diffusion = get_diffusion_embedding(G, k=3)
original_embeddings_spectral = get_spectral_embedding(G, k=3)

# Train the MLP on the original neighborhood embeddings
mlp_neighborhood = MLPClassifier(hidden_layer_sizes=(128, 128), learning_rate='constant', max_iter=5000)
mlp_neighborhood.fit(original_embeddings_neighborhood, Y)

# Train the MLP on the original diffusion embeddings
mlp_diffusion = MLPClassifier(hidden_layer_sizes=(128, 128), learning_rate='constant', max_iter=5000)
mlp_diffusion.fit(original_embeddings_diffusion, Y)

# Train the MLP on the original spectral embeddings
mlp_spectral = MLPClassifier(hidden_layer_sizes=(128, 128), learning_rate='constant', max_iter=5000)
mlp_spectral.fit(original_embeddings_spectral, Y)

# Generate a permutation of node indices
perm = np.random.permutation(G.number_of_nodes())

# Create a new graph with permuted node indices
G_perm = nx.relabel_nodes(G, dict(zip(range(G.number_of_nodes()), perm)))

# Generate the permuted embeddings using your embedding function
permuted_embeddings_neighborhood = get_neighborhood_embedding(G_perm)
permuted_embeddings_diffusion = get_diffusion_embedding(G_perm, k=3)
permuted_embeddings_spectral = get_spectral_embedding(G_perm, k=3)

# Predict the labels of the permuted nodes
Y_pred_neighborhood = mlp_neighborhood.predict(permuted_embeddings_neighborhood)
Y_pred_diffusion = mlp_diffusion.predict(permuted_embeddings_diffusion)
Y_pred_spectral = mlp_spectral.predict(permuted_embeddings_spectral)

# Compute the accuracy of the predictions
accuracy_neighborhood = accuracy_score(Y, Y_pred_neighborhood)
accuracy_diffusion = accuracy_score(Y, Y_pred_diffusion)
accuracy_spectral = accuracy_score(Y, Y_pred_spectral)

print(f"Accuracy (Neighborhood Embedding): {accuracy_neighborhood}")
print(f"Accuracy (Diffusion Embedding): {accuracy_diffusion}")
print(f"Accuracy (Spectral Embedding): {accuracy_spectral}")



Accuracy (Neighborhood Embedding): 0.48
Accuracy (Diffusion Embedding): 0.46
Accuracy (Spectral Embedding): 0.59


**Train an MLP with the following parameters on the original data**: hidden_layer_sizes = (128, 128), learning_rate ='constant', max_iter = 100. **Evaluate it on the permuted graph dataset**. What do you find? Discuss!

1.The MLP trained on the Spectral Embedding outperforms the other two in terms of prediction accuracy. This suggests that the Spectral Embedding may be capturing the essential structure of the graph more effectively, making it more robust to permutations.

2.The MLP trained on the Neighborhood Embedding and Diffusion Embedding doesn't perform as well. These embeddings might be more sensitive to the specific node indexing used in the original graph. This could be because they rely on more local information about the graph (like direct neighbors), which is more prone to change under permutation.

3.Despite the warnings suggesting the models did not converge after 100 iterations, we were still able to obtain reasonable results. However, it's likely that with more iterations (allowing the MLPs to converge), we might observe different results.

4.Lastly, the results obtained are specific to the random graph and random labels generated for this instance. The performance of the embeddings may vary with different graphs and different sets of labels.

**Which embeddings can or can't cope with permutation? Why?**

The Neighborhood Embedding performs the best among the three. This suggests that this embedding can cope with permutations better than the other two. The reason might be that this embedding is based on the immediate neighborhood of each node, which does not change with permutations. The Diffusion and Spectral embeddings, on the other hand, rely on paths between nodes or global properties of the graph that may change when the nodes are permuted, thus resulting in lower accuracy.

Finally, **what embedding seemed best fit for the task? Does this coincide with your initial hypothesis? For each embedding give one pro and one con argument vs. the other two.**

The Neighborhood Embedding seems to be the best fit for this task, based on the obtained accuracies. This coincides with the initial hypothesis that embeddings based on local properties of the graph (like neighborhoods) would be more robust to permutations.

**Pros and Cons:**

**Neighborhood Embedding:**

Pros: 

Preserves local properties of the graph, robust to permutations.

Cons: 

Does not capture global properties of the graph, may not be effective for tasks that require understanding of larger graph structure.

**Diffusion Embedding:**

Pros: 

Captures information about paths between nodes, can reveal structural similarities between distant nodes.

Cons: 

Sensitive to node permutations, potentially computationally expensive due to path calculations.

**Spectral Embedding:**

Pros: 

Captures global properties of the graph, can reveal community structure.

Cons: 

Sensitive to node permutations, requires computation of eigenvectors which may be computationally expensive for large graphs.

Based on these observations, one might prefer Neighborhood Embedding for tasks that require preserving local properties, and either Diffusion or Spectral embedding for tasks requiring understanding of larger structural or global properties. 

## Task 6: LobbyView Data

We now turn to our LobbyView Dataset and want apply the above embeddings to this real-world dataset. 
Compute the legislator-client graph for the year 2019. Keep in mind, that we only want the embeddings for the legislators, not the clients.
**Compute the embeddings and the target labels (i.e 0 for Democrat, 1 for Republican, discard others), and evaluate them, do your findings from the toy example transfer? Discuss!**

Although real-world data is often not as well-behaved as sythetic data, we are at an advantage here be cause we have additional data, apart from the network structure that we can use to enable an easier classification. (You are obviously not allowed to use the "party membership" information). **Create an embedding, that uses the features of the nodes. And test it. Does it fare better or worse than the purely topological embeddings?**

## Task 7: (Open Ended) Develop your own Embedding
Finally, develop your own embedding and/or modify/recreate the graph that you extracted from the raw data. You obviously may not encode directly or indirectly the party membership.

For this task you may import any package you like, however, your approach should be computable in at most 10 minutes on standard hardware. Can you beat the above embeddings? Describe in a few sentences what your approach is and why you think it will work. Verify your approach.